In [2]:
# set dependencies
import requests
import json
import urllib.request
import pandas as pd
from pandas import DataFrame

# National Parks Visitor table

In [3]:
##Extract CSVs into DataFrames
parks=pd.read_csv('dataFiles/parks.csv')

## Extract HTML to DataFrames
tables=pd.read_html('https://www.nps.gov/aboutus/visitation-numbers.htm', index_col=0)
table=pd.DataFrame(tables[1]).reset_index()

In [4]:
base_url='https://developer.nps.gov/api/v1'

    # API routes for API looping
nps_api_list=['/activities','/activities/parks','/alerts','/amenities','/amenities/parksplaces',
              '/amenities/parksvisitorcenters','/articles','/campgrounds','/events','/lessonplans','/newsreleases','/parks',
              '/passportstamplocations','/people','/places','/thingstodo','/topics','/topics/parks','/tours','/visitorcenters',
              '/webcams']

    # Name list for API looping
nps_api_names=['activities','activitiesparks','alerts','amenities','amenitiesparksplaces','amenitiesparksvisitorcenters',
               'articles','campgrounds','events','lessonplans','newsreleases','parks','passportstamplocations','people',
               'places','thingstodo','topics','topicsparks','tours','visitorcenters','webcams']

## Transform table and parks DataFrames to filter park dataframe
    # Merge table and parks dataframes to reduce parks to just top 10, create code list and convert list to lowercase
table_merged=table.merge(parks,'left',left_on='Park',right_on='Park Name')
nps_park_codes= parks['Park Code'].tolist()
top_ten_park_codes=table_merged['Park Code'].tolist()
top_ten_park_codes=[x.lower() for x in top_ten_park_codes]


In [5]:
activity_count=[]
for i in top_ten_park_codes:
    
##Load DataFrames into database
     #Extracting API data 
    url=f'{base_url}/parks?parkCode={i}&api_key=UIReZ9HdRnnl4YURWFi9bfWyLSCu9JPfEUBwlZIh'
    response=requests.get(url)
    top_ten=json.loads(response.content.decode('utf-8'))
     #Transform API data into list of activity counts
    top_ten_a=dict(top_ten['data'][0])    
    park_act=len([activity['name'] for activity in top_ten_a['activities']])
    activity_count.append(park_act)

# Greater Smoky Mountains

In [6]:
#Transform list into Dataframe
count_df=pd.DataFrame(activity_count,columns=['Number of Activities'])

In [7]:
count_df

,Number of Activities
0,36
1,53
2,22
3,34
4,53
5,33
6,31
7,46
8,54
9,27


In [8]:
# Get data table
tables=pd.read_html('https://www.nps.gov/aboutus/visitation-numbers.htm', index_col=0)
table=pd.DataFrame(tables[1]).reset_index()
table.rename(columns={'index':'Rank'})

,Rank,Park,Recreational Visits
0,1,Great Smoky Mountains National Park,12.1 million
1,2,Yellowstone National Park,"3,8 million"
2,3,Zion National Park,3.6 million
3,4,Rocky Mountain National Park,3.3 million
4,5,Grand Teton National Park,3.3 million
5,6,Grand Canyon National Park,2.9 million
6,7,Cuyahoga Valley National Park,2.8 million
7,8,Acadia National Park,2.7 million
8,9,Olympic National Park,2.5 million
9,10,Joshua Tree National Park,2.4 million


In [12]:
#Transform into single Dataframe
park_activity=table.merge(count_df,'left',left_index=True,right_index=True)

In [11]:
#Format index(rank) name
park_activity=park_activity.rename(columns={'index':'Rank'})
park_activity=park_activity.set_index('Rank')


KeyError: "None of ['Rank'] are in the columns"

In [15]:
park_activity.to_csv('dataFiles/activity_count.csv',encoding = 'utf-8',index=True)